In [1]:
import csv
from collections import namedtuple

# Define data structures
FogNode = namedtuple('FogNode', ['node_id', 'cpu', 'ram', 'storage'])
Service = namedtuple('Service', ['service_id', 'cpu', 'ram', 'storage', 'deadline'])

def read_data(fog_nodes_file, services_file):
    """Read data from CSV files and create lists of FogNode and Service objects"""
    fog_nodes = []
    with open(fog_nodes_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            node_id, cpu, ram, storage = row
            fog_nodes.append(FogNode(node_id, int(cpu), int(ram), int(storage)))

    services = []
    with open(services_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        for row in reader:
            service_id, cpu, ram, storage, deadline = row
            services.append(Service(service_id, int(cpu), int(ram), int(storage), int(deadline)))

    return fog_nodes, services

def place_services(fog_nodes, services):
    """Place services on fog nodes using bin-packing with deadline consideration"""
    # Sort services by decreasing order of deadlines
    services.sort(key=lambda s: s.deadline, reverse=True)

    placement = {node.node_id: [] for node in fog_nodes}
    missed_deadlines = []

    for service in services:
        placed = False
        for node in fog_nodes:
            # Check if node has sufficient resources
            if service.cpu <= node.cpu and service.ram <= node.ram and service.storage <= node.storage:
                # Calculate tentative completion time on this node
                completion_times = [s.deadline for s in placement[node.node_id]]
                tentative_completion_time = sum(s.cpu for s in placement[node.node_id]) + service.cpu
                if not completion_times or tentative_completion_time <= min(completion_times):
                    # Place service on this node
                    placement[node.node_id].append(service)
                    node.cpu -= service.cpu
                    node.ram -= service.ram
                    node.storage -= service.storage
                    placed = True
                    break

        if not placed:
            missed_deadlines.append(service)

    return placement, missed_deadlines

def generate_submission_file(placement, filename):
    """Generate the submission file in the required format"""
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['FNID', 'SERVICES'])
        for node_id, services in placement.items():
            service_ids = [service.service_id for service in services]
            if service_ids:
                service_ids_str = ', '.join(map(str, service_ids))
                writer.writerow([node_id, f'"{service_ids_str}"'])
            else:
                writer.writerow([node_id, '-'])

# Example usage
fog_nodes, services = read_data('fog_nodes.csv', 'services.csv')
placement, missed_deadlines = place_services(fog_nodes, services)
generate_submission_file(placement, 'submission.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'fog_nodes.csv'